In [30]:
from py2neo import Graph, NodeMatcher
import pandas as pd
from datetime import date

In [2]:
# connect to the graph -> it should be running already
graph = Graph(scheme="bolt", host="localhost", port=7687, auth=('neo4j', '12345'))

# clean up the database and PERMANENTLY delete all nodes and relationships
#graph.delete_all()
# clean up only a elements satisfying a specific MATCH
#query = "MATCH (n) WHERE n.name=.. delete n"
#graph.run(query)

##  ----------------  ##
# development milestone: add all functions to a class that holds the graph as a member
##  ----------------  ##

In [63]:
# utility functions ready to be used

class neo2BFMPolymer:
    """Python class to work on a neo4j graph data base with bfm polymer simulation data.
    
    The following node types are supported
    * Polymer - a class of polymer like linear chain, star polymer, ...
    * SimulationType - a group of simulation runs with a certain topic or setup 
      like linear polymer melt or star polymer solution
    * SimulationRun - a particular run of simulations with unique parameters 
    * Parameter - a simulation parameter like BoxSize that may be connected to multiple simulation runs
    * Result - a particular result of a simulation run that may be connected to an analyzer or a tool
    
    The following connectivity types are available
    * CONTAINS
    * INCLUDES
    * USES
    * ANALYZED
    
    Attributes:
        graph (py2neo.Graph): instance of the graph to access the neo4j database
        nodeMatcher (py2neo.NodeMatcher): matcher for the graph to evaluate nodes according to certain criteria.
        nodeMatcher (py2neo.RelationshipMatcher): matcher for the graph to evaluate relationships according to certain criteria.
    """
    def __init__(self, graph):
        """Constructor of the neo2BFMPolmyer class
        
        Parameters:
            graph (py2neo.Graph): instance of the graph to access the neo4j database
        """
        self.graph       = graph
        self.nodeMatcher = NodeMatcher(self.graph)
        #self.relaMatcher = RelationshipMatcher(self.graph)
        
        self.connectionType_polymerSimulationType = "CONTAINS" 
        self.connectionType_simTypeSimRun         = "INCLUDES"
        self.connectionType_simRunParameter       = "USES"
        self.connectionType_simRunResult          = "ANALYZED"
        
        self.nodeType_parameter      = "Parameter"
        self.nodeType_simulationRun  = "SimulationRun"
        self.nodeType_simulationType = "SimulationType"
        self.nodeType_polymer        = "Polymer"
        self.nodeType_result         = "Result"
    
    ### -------------- ### -------------- ### -------------- ###
    ### --------------    utility functions   -------------- ###
    ### -------------- ### -------------- ### -------------- ###
    def addNodeGeneral(self, nodeTypeName, nodeName):
        """Utility function adding a new node if it does not already exist.
        
        Paramters:
            nodeType (str): name of the node type
            nodeName (str): name property of the new node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        elementExists = self.graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodeName)).data()
        if (len(elementExists) > 0):
            print("WARNING: node of type {} with name {} already exists!".format(nodeTypeName,nodeName))
            return False
        else:
            query = "CREATE (pl:{} {{ name: \'{}\', createdOn: \'{}\' }} )".format(nodeTypeName, nodeName, date.today())
            self.graph.run(query)
            return True
        
        
    def addParameterSimulationRunGeneral(self,simulationRunName, parameterName, parameterValue):
        '''Utility function to connect a SimulationRun node with a Parameter node of name parameterName with the value parameterValue.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            parameterName (str): name property of the Parameter node
            parameterValue (str): value property of the parameter node
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeTypeSimRun = self.nodeType_simulationRun
        nodeNameSimRun = simulationRunName
        
        nodeTypeParameter  = self.nodeType_parameter
        nodeNameParameter  = parameterName
        nodeValueParameter = parameterValue
        
        connectionType = self.connectionType_simRunParameter
        
        simRunExist    = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeTypeSimRun,nodeNameSimRun))
        parameterExist = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" AND elem.value=\"{}\" return elem.value".format(nodeTypeParameter,nodeNameParameter,nodeValueParameter))
        
        if (simRunExist == nodeNameSimRun):
            # if box does not exist, create it
            if not (parameterExist == nodeValueParameter):
                query = "CREATE (bn:{} {{ name: \'{}\', createdOn: \'{}\', value: \'{}\' }} )".format(
                    nodeTypeParameter, nodeNameParameter, date.today(),nodeValueParameter)
                self.graph.run(query)
                
            query = '''MATCH (p:{}) WHERE p.name=\"{}\" AND p.value=\"{}\"
                           MATCH (s:{}) WHERE s.name=\"{}\"
                           MERGE (s)-[r:{}]-(p)
            '''.format(nodeTypeParameter,nodeNameParameter,nodeValueParameter,nodeTypeSimRun,nodeNameSimRun,connectionType)
            self.graph.run(query)
            return True
        
        else:
            print("WARNING: node of type {} with name {} does not exist!".format(nodeTypeSimRun,nodeNameSimRun))
            return False
    
    ### -------------- ### -------------- ### -------------- ###
    ### --------------   add node functions   -------------- ###
    ### -------------- ### -------------- ### -------------- ###
    def addNewPolymer(self, polymerName):
        """Adding a new Polymer node to the database.
        
        This function adds a node of type Polymer with the given name and a date tag to the database.
        If the database already contains a Polymer node with this name property, a warnig is printed
        and no create query is called to avoid node doubling.
        
        Paramters:
            polymerName (str): name property of the new Polymer node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        nodeTypeName     = self.nodeType_polymer
        nodePropertyName = polymerName
        
        return self.addNodeGeneral(nodeTypeName, nodePropertyName)
    
    
    def addNewSimulationType(self, simulationTypeName):
        """Adding a new SimulationType node to the database.
        
        A SimulationType node represents a certain simulation setup such as "linear polymer melt" or "single star polymer".
        This function adds a node of type SimulationType with the given name and a date tag to the database.
        If the database already contains a SimulationType node with this name property, a warnig is printed
        and no create query is called to avoid node doubling.
        
        Paramters:
            simulationTypeName (str): name property of the new SimulationType node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        nodeTypeName     = self.nodeType_simulationType
        nodePropertyName = simulationTypeName
        
        return self.addNodeGeneral(nodeTypeName, nodePropertyName)
    
            
    def addSimulationRunToSimulationType(self,simulationRunName,simulationTypeName):
        '''Add a new node of Type SimulationRun to the database.
        
        A SimulationRun node represents a particular simulation with all its paramters, the computation details
        and - if available - a path to the simulation files.
        The parameters are connected to the SimulationRun as self-contained nodes to find simulations
        with same paramters.
        A SimulationRun must belong to a SimulationType.
        This function adds a node of Type SimulationRun with the given name and a date tag to the database.
        If the database already contains a SimulationRun node with this name property, a warnig is printed
        and no create query is called to avoid node doubling.
        
        Paramters:
            simulationRunName (str): name property of the new SimulationRun node
            simulationTypeName(str): name property of the corresponding SimulationType node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        '''
        nodeTypeNameSimRun = self.nodeType_simulationRun
        nodeNameSimRun     = simulationRunName
        
        nodeTypeNameSimType = self.nodeType_simulationType
        nodeNameSimType     = simulationTypeName
        
        connectionType = self.connectionType_simTypeSimRun
        
        if self.addNodeGeneral(nodeTypeNameSimRun, nodeNameSimRun):
            query = '''MATCH (run:{}) WHERE run.name=\"{}\"
                       MATCH (type:{}) WHERE type.name=\"{}\"
                       MERGE (type)-[r:{}]->(run)
            '''.format(nodeTypeNameSimRun,nodeNameSimRun,nodeTypeNameSimType,nodeNameSimType,connectionType)
            self.graph.run(query)
            return True
        else:
            print("WARNING: node of type {} with name {} could not be added to {}".format(nodeTypeName,nodePropertyName,simulationTypeName))
            return False
    
    
    def connectSimulationToPolymer(self,polymerName,simulationName):
        """Connecting a node of type Polymer with a node of type SimulationType by a CONTAINS
        
        A connection of type CONTAINS is added between a Polymer node and a Simualtion node,
        heading in the direction of the polymer.
        The connection is only created if the nodes exist and the connection did not already exist.
        In this case, a warning is printed.
        
        Parameters:
            polymerName (str): name of the Polymer node
            simulationName (str): name of the SimulationType node
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        """
        nodeType1 = self.nodeType_polymer
        nodeName1 = polymerName
        nodeType2 = self.nodeType_simulationType
        nodeName2 = simulationName
        
        connectionType = self.connectionType_polymerSimulationType
        
        elementExists1 = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeType1,nodeName1))
        elementExists2 = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeType2,nodeName2))
        
        if (elementExists1 == nodeName1):
            if (elementExists2 == nodeName2):
                query = "MATCH( ({} {{ name: \'{}\'}}) <- [rel:{}] - ({} {{ name: \'{}\'}}) ) return rel".format(
                    nodeType1,nodeName1,connectionType,nodeType2,nodeName2)
                elementExists = self.graph.evaluate(query)
                if (elementExists == None):
                    query = '''MATCH (p:{}) WHERE p.name=\"{}\"
                               MATCH (s:{}) WHERE s.name=\"{}\"
                               MERGE (s)-[r:{}]->(p)
                    '''.format(nodeType1,nodeName1,nodeType2,nodeName2,connectionType)
                    self.graph.run(query)
                    return True
                
                else:
                    print("WARNING: connection of type {} between {} and {} already exist!".format(connectionType,nodeName1,nodeName2))
                    return False
                
            else:
                print("WARNING: node of type {} with name {} does not exist!".format(nodeType2,nodeName2))
                return False
            
        else:
            print("WARNING: node of type {} with name {} does not exist!".format(nodeType1,nodeName1))
            return False

        
    ### -------------- ### -------------- ### -------------- ###
    ### -------------- add parameter functions ------------- ###
    ### -------------- ### -------------- ### -------------- ###
    def addBoxSizeToSimulationRun(self,simulationRunName, boxX, boxY, boxZ):
        '''Connect a SimulationRun node with a Parameter node of name BoxSize with the given values.
        
        A parameter node with name BoxSize contains a value in the form "boxX x boxY x boxZ"
        used in FeatureBox of LeMonADE that can be connected to any SimulationRun node that uses this parameter.
        If the box node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            boxX (int): box dimension in x-direction
            boxY (int): box dimension in y-direction
            boxZ (int): box dimension in z-direction
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun = simulationRunName
        nodeNameBox    = "BoxSize"
        nodeValueBox   = "{} x {} x {}".format(boxX,boxY,boxZ)
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNameBox,nodeValueBox)
    
    def addPeriodicityToSimulationRun(self,simulationRunName, pX, pY, pZ):
        '''Connect a SimulationRun node with a Parameter node of name Periodicity with the given values.
        
        A parameter node with name Periodicity contains a value in the form "pX x pY x pZ" encoded as bools ("True", "False")
        used in FeatureBox of LeMonADE that can be connected to any SimulationRun node that uses this parameter.
        If the Periodicity node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            pX (bool): periodicity in x-direction (True = periodic, False = non-periodic)
            pY (bool): periodicity in y-direction (True = periodic, False = non-periodic)
            pZ (bool): periodicity in z-direction (True = periodic, False = non-periodic)
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun       = simulationRunName
        nodeNamePeriodicity  = "Periodicity"
        nodeValuePeriodicity = "{} x {} x {}".format(pX,pY,pZ)
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNamePeriodicity,nodeValuePeriodicity)
        
        
    def addNNInteractionToSimulationRun(self,simulationRunName, epsilon):
        '''Connect a SimulationRun node with a Parameter node of name NNInteraction with the given value.
        
        A parameter node with name NNInteraction contains a value epsilon set in FeatureNNInteraction of LeMonADE
        that can be connected to any SimulationRun node that uses this parameter.
        If the NNInteraction node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            epsilon (float): interaction energy per nearest neighbor lattice point
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun = simulationRunName
        nodeNameNNInt  = "NNInteraction"
        nodeValueNNInt = "{0:.3f}".format(epsilon) # is this precision the best choice?
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNameNNInt,nodeValueNNInt)
    
    
    def addLinearChainLengthToSimulationRun(self,simulationRunName, N):
        '''Connect a SimulationRun node with a Parameter node of name LinearChainLength with the given value.
        
        A parameter node with name LinearChainLength contains a value N as common parameter
        that can be connected to any SimulationRun node that uses this parameter.
        If the LinearChainLength node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            n (int): number of BFM units of a linear chain
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun       = simulationRunName
        nodeNameChainLength  = "LinearChainLength"
        nodeValueChainLength = "{}".format(N)
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNameChainLength,nodeValueChainLength)
    
    
    def addDendrimerGenerationToSimulationRun(self,simulationRunName, G):
        '''Connect a SimulationRun node with a Parameter node of name DendrimerGeneration with the given value.
        
        A parameter node with name DendrimerGeneration contains a value G
        that can be connected to any SimulationRun node that uses this parameter.
        If the DendrimerGeneration node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            G (int): number of generations in a dendrimer molecule
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun           = simulationRunName
        nodeNameDendrGeneration  = "DendrimerGeneration"
        nodeValueDendrGeneration = "{}".format(G)
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNameDendrGeneration,nodeValueDendrGeneration)
    
    
    def addDendrimerSpacerLengthToSimulationRun(self,simulationRunName, S):
        '''Connect a SimulationRun node with a Parameter node of name DendrimerSpacerLength with the given value.
        
        A parameter node with name DendrimerSpacerLength contains a value S
        that can be connected to any SimulationRun node that uses this parameter.
        If the DendrimerSpacerLength node does not exist, it is created and then connected to the SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            S (int): number of BFM units between two branching points of a dendrimer
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeNameSimRun       = simulationRunName
        nodeNameDendrSpacer  = "DendrimerSpacerLength"
        nodeValueDendrSpacer = "{}".format(S)
        
        return self.addParameterSimulationRunGeneral(nodeNameSimRun,nodeNameDendrSpacer,nodeValueDendrSpacer)
    
    def addResultRadiusOfGyration(self,simulationRunName, Rg):
        '''Adding a Result node with a single value or an array of the radius of gyration of a particular SimulationRun.
        
        Parameters:
            simulationRunName (str): name of the SimulationRun node
            Rg (float,array): radius of gyration calculated for the conformations in the simulation run.
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists 
                              or SimulationRun node does not exist
        '''
        nodeTypeSimRun = self.nodeType_simulationRun
        nodeNameSimRun = simulationRunName
        nodeTypeRg     = self.nodeType_result
        nodeNameRg     = "RadiusOfGyration"
        nodeValueRg    = str(Rg) # this needs to be improved!
        connectionType = self.connectionType_simRunResult
        
        simExQuery = "MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeTypeSimRun,nodeNameSimRun)
        simRunExist = self.graph.evaluate(simExQuery)
        resExQuery = "MATCH (simRun:{} {{name:\"{}\"}})-[rel:{}]-(n:{} {{name:\"{}\"}}) return n.name".format(nodeTypeSimRun,nodeNameSimRun,connectionType,nodeTypeRg,nodeNameRg)
        resultExist = self.graph.evaluate(resExQuery)
        
        if (simRunExist == nodeNameSimRun):

            if not (resultExist == nodeNameRg):
                # create result node with value of Rg
                query = "CREATE (rslt:{} {{ name: \'{}\', createdOn: \'{}\', value: \'{}\' }} )".format(
                    nodeTypeRg, nodeNameRg, date.today(), nodeValueRg)
                self.graph.run(query)
                
                # merge simulationRun node with result node
                query = '''MATCH (result:{}) WHERE result.name=\"{}\" AND result.value=\"{}\"
                           MATCH (simRun:{}) WHERE simRun.name=\"{}\"
                           MERGE (simRun)-[r:{}]->(result)
                '''.format(nodeTypeRg,nodeNameRg,nodeValueRg,nodeTypeSimRun,nodeNameSimRun,connectionType)
                print(query)
                self.graph.run(query)
                
                return True
            
            else:
                print("WARNING: node of type {} with name {} is already connected to {} node {}!".format(nodeTypeSimRun,nodeNameSimRun,nodeTypeRg,nodeNameRg))
                return False
        
        else:
            print("WARNING: node of type {} with name {} does not exist!".format(nodeTypeSimRun,nodeNameSimRun))
            return False

In [68]:
# test the functions

# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

myNeoPolymers = neo2BFMPolymer(graph)
print("add linear chain as polymer: ", myNeoPolymers.addNewPolymer("Linear Chain"))
print("add dendrimers as polymer: ", myNeoPolymers.addNewPolymer("Dendrimer"))
print("add star polymer as polymer: ", myNeoPolymers.addNewPolymer("Star Polymer"))
print("add comb polymer as polymer: ", myNeoPolymers.addNewPolymer("Comb Polymer"))

print("add Linear Polymer Melt as SimulationType: ", myNeoPolymers.addNewSimulationType("Linear Polymer Melt"))
print("add Single Linear Chain as SimulationType: ", myNeoPolymers.addNewSimulationType("Single Linear Chain"))
print("add Single Dendrimer as SimulationType: ", myNeoPolymers.addNewSimulationType("Single Dendrimer"))
print("add Mixture Dendrimer and Linear Chains as Simulation: ", myNeoPolymers.addNewSimulationType("Mixture Dendrimer and Linear Chains"))

print("connect polymer and Melt", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Linear Polymer Melt"))
print("connect polymer and single chain", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Single Linear Chain"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Single Dendrimer"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Mixture Dendrimer and Linear Chains"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Mixture Dendrimer and Linear Chains"),"\n")

print("add SimulationRun single dendrimers run 1", myNeoPolymers.addSimulationRunToSimulationType("Single Dendrimers Solvent Quality","Single Dendrimer"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Dendrimers Solvent Quality", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Dendrimers Solvent Quality", True, True, True))
print("connect DendrimerGeneration", myNeoPolymers.addDendrimerGenerationToSimulationRun("Single Dendrimers Solvent Quality", 5))
print("connect DendrimerSpacerLength", myNeoPolymers.addDendrimerSpacerLengthToSimulationRun("Single Dendrimers Solvent Quality", 2))
print("connect NNInteraction", myNeoPolymers.addNNInteractionToSimulationRun("Single Dendrimers Solvent Quality", 0.8),"\n")

print("add SimulationRun single chains run 1", myNeoPolymers.addSimulationRunToSimulationType("Single Linear Chain At Wall","Single Linear Chain"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Linear Chain At Wall", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Linear Chain At Wall", True, True, False))
print("connect ChainLength", myNeoPolymers.addLinearChainLengthToSimulationRun("Single Linear Chain At Wall", 128))
print("add Result node Rg",myNeoPolymers.addResultRadiusOfGyration("Single Linear Chain At Wall",46.8),"\n")

add linear chain as polymer:  True
add dendrimers as polymer:  True
add star polymer as polymer:  True
add comb polymer as polymer:  True
add Linear Polymer Melt as SimulationType:  True
add Single Linear Chain as SimulationType:  True
add Single Dendrimer as SimulationType:  True
add Mixture Dendrimer and Linear Chains as Simulation:  True
connect polymer and Melt True
connect polymer and single chain True
connect dendrimers True
connect dendrimers True
connect dendrimers True 

add SimulationRun single dendrimers run 1 True
connect BoxSize True
connect Periodicity True
connect DendrimerGeneration True
connect DendrimerSpacerLength True
connect NNInteraction True 

add SimulationRun single chains run 1 True
connect BoxSize True
connect Periodicity True
connect ChainLength True
MATCH (elem:SimulationRun) WHERE elem.name="Single Linear Chain At Wall" return elem.name 
 Single Linear Chain At Wall 
 Single Linear Chain At Wall
MATCH (simRun:SimulationRun {name:"Single Linear Chain At Wal

In [66]:
#this should be represented finally: codmuc_micelle_g3_s2_m12_l2_e0.8_b128.bfm
myNeoPolymers.addResultRadiusOfGyration("Single Linear Chain At Wall",46.8)

MATCH (elem:SimulationRun) WHERE elem.name="Single Linear Chain At Wall" return elem.name 
 Single Linear Chain At Wall 
 Single Linear Chain At Wall
MATCH (simRun:SimulationRun {name:"Single Linear Chain At Wall"})-[rel:ANALYZED]-(n:Result {name:"RadiusOfGyration"}) return n.name 
 RadiusOfGyration


False

In [ ]:
## Here some try and error snippets I want to keep for the moment follow:

In [98]:
# Create the generic type 

nodeTypeName = "Simulation Type"
nodePropertyName = "Comblike Polymer"

elementExists = graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodePropertyName)).data()
print(elementExists)
if (len(elementExists) > 0):
    print("WARNING: node of type {} with name {} already exists!".format(nodeTypeName,nodePropertyName))
else:
    query = "CREATE (pl:{} {{ name: \'{}\', createdOn: \'{}\' }} )".format(nodeTypeName, nodePropertyName, date.today())
    print(query)
    graph.run(query)

#query = "MATCH (pD:Polymer) return pD"
#result = graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodePropertyName)).data()
#simTypeAlias = Node("Polymer", name = "linear chains")
#CREATE (n:Person { name: 'Andy', title: 'Developer' })
# polymerLinearChain = graph.run("CREATE (pl:Polymer { name: 'Linear Chains'} )")

#polymerLinearChain = graph.run("CREATE (pl:Polymer { name: 'Linear Chains'} )")

[{'elem': (_8:Polymer {createdOn: '2020-06-25', name: 'Comblike Polymer'})}, {'elem': (_9:Polymer {createdOn: '2020-06-25', name: 'Comblike Polymer'})}]


In [23]:
#polymerDendrimer, = graph.run("CREATE (a:Polymer { name: 'Dendrimer'} )")
polymerDendrimer = graph.run("CREATE (pd:Polymer { name: 'Dendrimer'} )")

In [74]:
simDIPS = graph.run("CREATE (a:Simulation {name: 'Mixture of Dendrimers and Linear Polymer Chains'})").data()
print(simDIPS)

[]


In [43]:
#access a single node
query = "MATCH (pD:Polymer) WHERE pD.name='Dendrimer' return pD"
#query = "MATCH (pD:Polymer) return pD"
result = graph.evaluate(query)
# result if of type  py2neo.data.Node
print("print (only!) the first element of MATCH query:\n", result)
print("\naccess properties with [] operator:\t", result["name"])

print (only!) the first element of MATCH query:
 None


TypeError: 'NoneType' object is not subscriptable

In [26]:
query = "MATCH (pD:Polymer) return pD"
result = graph.run(query).data()
print(result)
print(pd.DataFrame(result))

[{'pD': (_2:Polymer {createdOn: '2020-06-30', name: 'Linear Chain'})}, {'pD': (_3:Polymer {createdOn: '2020-06-30', name: 'Dendrimer'})}, {'pD': (_4:Polymer {createdOn: '2020-06-30', name: 'Star Polymer'})}, {'pD': (_5:Polymer {createdOn: '2020-06-30', name: 'Comb Polymer'})}]
                                                  pD
0  {'name': 'Linear Chain', 'createdOn': '2020-06...
1   {'name': 'Dendrimer', 'createdOn': '2020-06-30'}
2  {'name': 'Star Polymer', 'createdOn': '2020-06...
3  {'name': 'Comb Polymer', 'createdOn': '2020-06...


In [29]:
rgList=[4.5,4.6,5.0]
print(rgList, str(rgList))

[4.5, 4.6, 5.0] [4.5, 4.6, 5.0] [4.5 4.6 5. ]


In [65]:
# Add a new relationship between existing nodes, here with relation type "CONTAINS":

# MATCH (p:Polymer) WHERE p.name="Dendrimer"
# MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
# MERGE (s)-[r:CONTAINS]->(p)

query = '''
MATCH (p:Polymer) WHERE p.name="Linear Chains"
MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
MERGE (s)-[r:CONTAINS]->(p)
'''
graph.run(query)

query = '''
MATCH (p:Polymer) WHERE p.name="Dendrimer"
MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
MERGE (s)-[r:CONTAINS]->(p)
'''
graph.run(query)

py2neo.data.Node

In [53]:
print(res)

In [75]:
query = "MATCH (n:Simulation) WHERE n.name='Mixture of Dendrimers and Linear Polymer Chains' delete n"
graph.run(query)

ClientError: ConstraintValidationFailed: Cannot delete node<2>, because it still has relationships. To delete this node, you must first delete its relationships.